In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
import seaborn as sns
#https://www.kaggle.com/code/davidcoxon/football-transfer-market-eda-basic-modelling/notebook#Observation-on-revised-model

In [2]:
appearances_df = pd.read_csv('https://query.data.world/s/uwuo6sxfqlzsbm6jhalo45win3eut4?dws=00000')
clubs_df = pd.read_csv('https://query.data.world/s/qaze23ezd5erez5yxybgepvhj3e3af?dws=00000')
club_games_df = pd.read_csv('https://query.data.world/s/cnbbhsthqv35pjvcnq2ial47jsgxkh?dws=00000')
competitions_df = pd.read_csv('https://query.data.world/s/o6waa35f7hgvesipsqhh2ir7ymav3y?dws=00000')
games_df = pd.read_csv('https://query.data.world/s/l4j3ct5jctslxloghv5plnmgnn3xww?dws=00000')
game_events_df = pd.read_csv('https://query.data.world/s/np7xbqq6hjfxaeoei3n2tc5orqliia?dws=00000')
players_df = pd.read_csv('https://query.data.world/s/gtwcwmx2rpe42p4a3upe4zh2yvaj2m?dws=00000')
player_valuations_df = pd.read_csv('https://query.data.world/s/mlqges6k3mpachaluntpm6gbxnj37o?dws=00000')


## Data Processing

In [3]:
#calculate the age of players
players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])

#drop players without date of birth
players_df = players_df[players_df['date_of_birth'].isnull() == False]

today = datetime.now()

players_df.loc[:, 'age'] = (today - players_df['date_of_birth']).apply(lambda x: x.days) / 365.25
players_df.loc[:, 'age'] = players_df['age'].round().astype(int)


In [4]:
#Calculate remaining contract value of each player
players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])

#get rid of players with no expiration rdate
players_df = players_df[players_df['contract_expiration_date'].isnull() == False]

players_df['term_days_remaining'] = (players_df['contract_expiration_date'] - today).apply(lambda x: x.days)

In [5]:
# add year to player valuations
player_valuations_df['datetime']=pd.to_datetime(player_valuations_df['datetime'])
player_valuations_df['year']=player_valuations_df['datetime'].dt.year



In [6]:
#add yar to player appearances
appearances_df['datetime'] = pd.to_datetime(appearances_df['date'], format = "%Y-%m-%d")
appearances_df['year'] = appearances_df['datetime'].dt.year

In [7]:
#add position to player valuations
position_df=players_df.copy()
position_df=position_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth', 
                              'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'market_value_in_eur', 
                              'highest_market_value_in_eur', 'agent_name', 'contract_expiration_date', 'current_club_domestic_competition_id', 
                              'first_name', 'last_name', 'player_code', 'image_url', 'last_season', 'url', 'age'], axis=1)

player_valuations_df = player_valuations_df.merge(
position_df, left_on = 'player_id', right_on = 'player_id')

In [8]:
#add position to appearances
#add position to appearances
value_df=players_df.copy()
value_df=value_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth',
                        'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'sub_position',
                        'highest_market_value_in_eur', 'agent_name', 'contract_expiration_date', 
                        'current_club_domestic_competition_id', 'first_name', 'last_name', 'player_code', 
                        'image_url', 'last_season', 'url', 'age'], axis=1)
appearances_df1=value_df.merge(appearances_df, left_on='player_id', right_on='player_id')

## Combining PLayer Data

In [9]:
merged_players_df=players_df.drop(['current_club_id', 'city_of_birth', 'date_of_birth','first_name',
                                   'last_name', 'player_code', 'image_url', 'url'], axis=1)

In [10]:
#next look at clubs_df
merged_players_df = merged_players_df.reindex(columns = merged_players_df.columns.tolist() + 
                                              ['club_value','squad_size','goals','goals_2023','games_2023',
                                               'assists_2023','minutes_played_2023','goals_against_2023',
                                               'goals_for_2023','clean_sheet_2023'])


In [11]:
for player_id in merged_players_df.player_id.unique():
    club_id = players_df.current_club_id[players_df.player_id == player_id].iloc[0]
    club_value = clubs_df.total_market_value[clubs_df.club_id == club_id].iloc[0]
    squad_size = clubs_df.squad_size[clubs_df.club_id == club_id].iloc[0]
    
    try:
        merged_players_df.loc[merged_players_df.player_id == player_id, 'club_value'] = int(club_value)
    except:
        merged_players_df.loc[merged_players_df.player_id == player_id, 'club_value'] = 'NaN'
    
    merged_players_df.loc[merged_players_df.player_id == player_id, 'squad_size'] = int(squad_size)
    


In [12]:
#sort column order
columns=['player_id','games_2023','minutes_played_2023','goals_2023','assists_2023',
         'goals_against_2023','goals_for_2023','clean_sheet_2023','name','position',
         'sub_position','last_season','foot','height_in_cm','age','country_of_citizenship',
         'country_of_birth','current_club_name','club_value','squad_size',
         'current_club_domestic_competition_id','agent_name','contract_expiration_date',
         'term_days_remaining','market_value_in_eur','highest_market_value_in_eur']
merged_players_df=merged_players_df[columns]

In [13]:
#merge games and appearances
games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')

In [14]:
season = 2022

In [15]:
#create a function to collate player stats
def player_stats(player_id, season, df):
    df = games_and_appearances_df[games_and_appearances_df['player_id'] == player_id]
    df =  df[ df['season'] == season]    
    if (df.shape[0] == 0):
        Out = [(np.nan, season,0,0,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','season','goals','games',
                                                     'assists','minutes_played','goals_for','goals_against',
                                                     'clean_sheet','yellow_cards','red_cards'])
        return out_df    
    else:       
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
            else 0 if row['goals_against'] > 0
            else np.nan, axis=1)
        df = df.groupby(['player_id',"season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                      'assists': 'sum', 'minutes_played' : 'sum', 
                                                                    'goals_for' : 'sum',
                                                                      'goals_against' : 'sum', 'clean_sheet' : 
                                                                    'sum','yellow_cards':'sum','red_cards':'sum'})
        out_df = df.rename(columns={'game_id': 'games'})
        return out_df
#print(player_stats(67064, 2014, games_and_appearances_df)) #test function works.



In [16]:
#iterate through players
for index in merged_players_df.index:
    id = merged_players_df.loc[index][0]
    #print(id)
    name = merged_players_df.loc[index][1]
    stats = player_stats(id, season, games_and_appearances_df)
    try:
        merged_players_df.at[index,'games_{}'.format(2023)]= stats['games'][0]
        merged_players_df.at[index,'goals_{}'.format(2023)]= stats['goals'][0]
        merged_players_df.at[index,'assists_{}'.format(2023)]= stats['assists'][0]
        merged_players_df.at[index,'minutes_played_{}'.format(2023)]= stats['minutes_played'][0]
        merged_players_df.at[index,'goals_for_{}'.format(2023)]= stats['goals_for'][0]
        merged_players_df.at[index,'goals_against_{}'.format(2023)]= stats['goals_against'][0]
        merged_players_df.at[index,'clean_sheet_{}'.format(2023)]= stats['clean_sheet'][0]
        merged_players_df.at[index,'yellow_cards_{}'.format(2023)]= stats['yellow_cards'][0]
        merged_players_df.at[index,'red_cards_{}'.format(2023)]= stats['red_cards'][0]
    except:
        #print(id)
        n=n+1     
print('appearance,goal and card data merged')
print(merged_players_df.info());#print(merged_players_df.describe())

appearance,goal and card data merged
<class 'pandas.core.frame.DataFrame'>
Index: 17938 entries, 21 to 28458
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   player_id                             17938 non-null  int64         
 1   games_2023                            17938 non-null  float64       
 2   minutes_played_2023                   17938 non-null  float64       
 3   goals_2023                            17938 non-null  float64       
 4   assists_2023                          17938 non-null  float64       
 5   goals_against_2023                    17938 non-null  float64       
 6   goals_for_2023                        17938 non-null  float64       
 7   clean_sheet_2023                      17938 non-null  float64       
 8   name                                  17938 non-null  object        
 9   position                              1

In [17]:
#get rid of players who have negative term_days_remaining
merged_players_df = merged_players_df[merged_players_df.term_days_remaining >= 0]

## Narrow down to big 5 leagues

In [18]:
big_five = ['GB1', 'L1', 'IT1', 'FR1', 'ES1']
merged_players_df = merged_players_df[merged_players_df['current_club_domestic_competition_id'].isin(big_five)]

## Change Specific String Columns to Numeric Columns

In [19]:
#convert right and left foot to numeric column
merged_players_df.loc[:, 'foot'] = merged_players_df['foot'].map({'right' :1, 'left' :0})

In [20]:
#convert position to numeric column
merged_players_df.loc[:, 'position'] = merged_players_df['position'].map({
    'Goalkeeper': 0,
    'Midfield' : 1,
    'Defender': 2,
    'Attack': 3,
    'Missing': 4
})

In [21]:
#grab subpositions and create mapping
subpositions = merged_players_df.sub_position.unique()

subpos_map = {}

for i in range(len(subpositions)):
    subpos_map[subpositions[i]] = i

In [22]:
#substitute numeric values for subposition
merged_players_df.loc[:, 'sub_position'] = merged_players_df['sub_position'].map(subpos_map)

In [23]:
league_map = {}
for i in range(len(big_five)):
    league_map[big_five[i]] = i

In [24]:
#substitute numeric values for league
merged_players_df.loc[:, 'current_club_domestic_competition_id'] = merged_players_df['current_club_domestic_competition_id'].map(league_map)

In [25]:
numeric_cols = ['games_2023', 'minutes_played_2023', 'goals_2023', 'assists_2023', 'goals_against_2023',
               'goals_for_2023', 'clean_sheet_2023', 'position', 'sub_position', 'foot', 'height_in_cm',
               'age', 'term_days_remaining', 'market_value_in_eur', 'highest_market_value_in_eur', 
               'yellow_cards_2023', 'red_cards_2023', 'current_club_domestic_competition_id']

In [26]:
num_merged = merged_players_df[numeric_cols]
num_merged = num_merged.dropna()

### Narrow to Big 5 Leagues only and Export

In [27]:
num_merged.to_csv('data/2022_big5.csv', index = False)